In [ ]:
!pip install transformers
from ade20k_id2label import CONFIG as CONFIG_ADE20K_ID2LABEL
import pycocotools.mask as maskUtils
import torch

In [ ]:
import torch.nn.functional as F
import numpy as np

def segformer_func(image, processor, model, rank):
    h, w, _ = image.shape
    inputs = processor(images=image, return_tensors="pt").to(rank)
    outputs = model(**inputs)
    logits = outputs.logits
    logits = F.interpolate(logits, size=(h, w), mode='bilinear', align_corners=True)
    predicted_semantic_map = logits.argmax(dim=1).squeeze(0)
    return predicted_semantic_map

In [ ]:
import sys
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

!mkdir ./weights

%cd /content/weights
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
import os

CHECKPOINT_PATH = os.path.join("/content", "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-c64vq0es
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-c64vq0es
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36589 sha256=32ebbb9cc143d9ffd2be022acc2c4d01e8ca8c7e9fe77f8e861f2528691e502d
  Stored in directory: /tmp/pip-ephem-wheel-cache-uen8y4o3/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything
/content/weights
/content/weights/sam_vit_h_4b8939.pth ; exist: True


In [ ]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"
sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)
mask_branch_model = SamAutomaticMaskGenerator(sam)

In [ ]:
from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation
semantic_branch_processor = SegformerFeatureExtractor.from_pretrained(
                "nvidia/segformer-b5-finetuned-ade-640-640")
semantic_branch_model = SegformerForSemanticSegmentation.from_pretrained(
                "nvidia/segformer-b5-finetuned-ade-640-640").to(0)
id2label = CONFIG_ADE20K_ID2LABEL

/usr/local/lib/python3.10/dist-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/segformer/image_processing_segformer.py:99: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [ ]:
IMAGE_PATH = "/content/1.jpg"
class_names = []

In [ ]:
import cv2
img = cv2.imread(IMAGE_PATH)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
anns = {'annotations': mask_branch_model.generate(img)}
h, w, _ = img.shape
class_ids = segformer_func(img, semantic_branch_processor, semantic_branch_model, 0)
semantc_mask = class_ids.clone()
anns['annotations'] = sorted(anns['annotations'][:5], key=lambda x: x['area'], reverse=True)

In [ ]:
for ann in anns['annotations']:
        valid_mask = torch.tensor(ann['segmentation']).bool()
        # get the class ids of the valid pixels
        propose_classes_ids = class_ids[valid_mask]
        print("propose_classes_ids = ",propose_classes_ids)
        num_class_proposals = len(torch.unique(propose_classes_ids))
        print("num_class_proposals = ",num_class_proposals)
        if num_class_proposals == 1:
            semantc_mask[valid_mask] = propose_classes_ids[0]
            ann['class_name'] = id2label['id2label'][str(propose_classes_ids[0].item())]
            ann['class_proposals'] = id2label['id2label'][str(propose_classes_ids[0].item())]
            class_names.append(ann['class_name'])
            # bitmasks.append(maskUtils.decode(ann['segmentation']))
            continue
        top_1_propose_class_ids = torch.bincount(propose_classes_ids.flatten()).topk(1).indices
        top_1_propose_class_names = [id2label['id2label'][str(class_id.item())] for class_id in top_1_propose_class_ids]

        
        semantc_mask[valid_mask] = top_1_propose_class_ids
        ann['class_name'] = top_1_propose_class_names[0]
        ann['class_proposals'] = top_1_propose_class_names[0]
        class_names.append(ann['class_name'])
        # bitmasks.append(maskUtils.decode(ann['segmentation']))

        del valid_mask
        del propose_classes_ids
        del num_class_proposals
        del top_1_propose_class_ids
        del top_1_propose_class_names

propose_classes_ids =  tensor([5, 5, 5,  ..., 0, 0, 0], device='cuda:0')
num_class_proposals =  2
propose_classes_ids =  tensor([5, 5, 5,  ..., 5, 5, 5], device='cuda:0')
num_class_proposals =  1
propose_classes_ids =  tensor([5, 5, 5,  ..., 0, 0, 0], device='cuda:0')
num_class_proposals =  2
propose_classes_ids =  tensor([8, 8, 8,  ..., 8, 8, 8], device='cuda:0')
num_class_proposals =  2
propose_classes_ids =  tensor([8, 8, 8,  ..., 8, 8, 8], device='cuda:0')
num_class_proposals =  1


In [ ]:
sematic_class_in_img = torch.unique(semantc_mask)
semantic_bitmasks, semantic_class_names = [], []
anns['semantic_mask'] = {}
for i in range(len(sematic_class_in_img)):
        class_name = id2label['id2label'][str(sematic_class_in_img[i].item())]
        class_mask = semantc_mask == sematic_class_in_img[i]
        class_mask = class_mask.cpu().numpy().astype(np.uint8)
        semantic_class_names.append(class_name)
        semantic_bitmasks.append(class_mask)
        anns['semantic_mask'][str(sematic_class_in_img[i].item())] = maskUtils.encode(np.array((semantc_mask == sematic_class_in_img[i]).cpu().numpy(), order='F', dtype=np.uint8))
        anns['semantic_mask'][str(sematic_class_in_img[i].item())]['counts'] = anns['semantic_mask'][str(sematic_class_in_img[i].item())]['counts'].decode('utf-8')

In [ ]:
cd ..

/content


In [ ]:
!mkdir ./masks
!mkdir ./masked_img

In [ ]:
import cv2
from PIL import Image as im
def generate_masks(anns):
  i = 0
  for ann in anns['annotations']:
    data = im.fromarray(ann['segmentation'])
    data.save(f'/content/masks/mask{i}_{ann["class_name"]}_{IMAGE_PATH.split("/")[-1]}')
    i = i+1
    if i == 5:
      break

In [ ]:
def masked_img():
  masks = os.listdir("./masks")
  j = 0
  for i in masks:
    img = cv2.imread(IMAGE_PATH)
    mask = cv2.imread(f"/content/masks/{i}",0)
    res = cv2.bitwise_and(img,img,mask = mask)
    src = res
    tmp = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    _, alpha = cv2.threshold(tmp, 0, 255, cv2.THRESH_BINARY)
    b, g, r = cv2.split(src)
    rgba = [b, g, r, alpha]
    dst = cv2.merge(rgba, 4)
    cv2.imwrite(f'/content/masked_img/masked_{i}', dst)


In [ ]:
generate_masks(anns)
masked_img()

***Damage detection model*** 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/models_yolo/YOLOv8-3.pt  show=True conf=0.5 source=/content/masked_img
#!yolo task=detect mode=predict model=/content/drive/MyDrive/models_yolo/YOLOv8.pt source=/content/1379.jpg show=True


WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Ultralytics YOLOv8.0.89 🚀 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs

image 1/5 /content/masked_img/masked_mask0_wall_12.jpg: 288x416 1 damage, 62.2ms
image 2/5 /content/masked_img/masked_mask1_ceiling_12.jpg: 288x416 1 damage, 8.3ms
image 3/5 /content/masked_img/masked_mask2_ceiling_12.jpg: 288x416 (no detections), 8.2ms
image 4/5 /content/masked_img/masked_mask3_wall_12.jpg: 288x416 1 damage, 8.2ms
image 5/5 /content/masked_img/masked_mask4_door_12.jpg: 288x416 (no detections), 8.2ms
Speed: 1.0ms preprocess, 19.0ms inference, 17.2ms postprocess per image at shape (1, 3, 416, 416)
Results saved to runs/detect/predict7


In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.89 🚀 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.3/166.8 GB disk)


In [ ]:
!yolo predict model=yolov8n.pt source='https://ultralytics.com/images/zidane.jpg'

100% 6.23M/6.23M [00:00<00:00, 71.7MB/s]
Ultralytics YOLOv8.0.89 🚀 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

100% 165k/165k [00:00<00:00, 9.71MB/s]
image 1/1 /content/zidane.jpg: 384x640 2 persons, 1 tie, 111.0ms
Speed: 14.5ms preprocess, 111.0ms inference, 379.3ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict


In [ ]:
import shutil
masks = os.listdir("./masks")
for i in masks:
  file_path = os.path.join("masks", i)
  try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
  except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))
    

In [ ]:
import shutil
masked = os.listdir("./masked_img")
for i in masked:
  file_path = os.path.join("masked_img", i)
  try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
  except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

**DAMAGE DETECTION MODEL YOLOv5**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
import torch
import os
from IPython.display import Image, clear_output  # to display images
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 15637, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 15637 (delta 123), reused 173 (delta 100), pack-reused 15393
Receiving objects: 100% (15637/15637), 14.58 MiB | 22.83 MiB/s, done.
Resolving deltas: 100% (10652/10652), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00
Setup complete. Using torch 2.0.0+cu118 (Tesla T4)


In [ ]:
!python /content/yolov5/detect.py --weights /content/drive/MyDrive/models_yolo/bestYOLOv5.pt --img 512 --conf 0.4 --source /content/test

detect: weights=['/content/drive/MyDrive/models_yolo/bestYOLOv5.pt'], source=/content/test, data=yolov5/data/coco128.yaml, imgsz=[512, 512], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: /content/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-162-gc3e4e94 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/61 /content/test/27002.jpg: 384x512 (no detections), 39.7ms
image 2/61 /content/test/27007.jpg: 384x512 1 damage, 7.5ms
image 3/61 /content/test/27008.jpg: 384x512 (no detections), 7.6ms
image 4/61 /content/test/27014.jpg: 384x512 (no detections), 

In [ ]:
imag = os.listdir("/content/yolov5/runs/detect/exp3")

In [ ]:
from google.colab import files
for i in imag:
 files.download(f'/content/yolov5/runs/detect/exp3/{i}')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>